# Import Libraries

In [2]:
import os
# disable tensorflow log level infos
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' # show only errors

In [3]:
from typing import Tuple

from absl import app
from absl import flags
import numpy as np
import pyglove as pg
import tensorflow as tf

# Check GPU Availability

In [4]:
## restrict memory growth -------------------
import tensorflow as tf
physical_devices = tf.config.list_physical_devices('GPU') 
try:
    gpu_0 = physical_devices[0]
    tf.config.experimental.set_memory_growth(gpu_0, True) 
    #tf.config.experimental.set_virtual_device_configuration(gpu_0, [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=6500)])
    print(' ==> Restrict GPU memory growth: True')
except: 
    raise Exception("Invalid device or cannot modify virtual devices once initialized.")
## restrict memory growth ------------------- 

 ==> Restrict GPU memory growth: True


# Utilitary Functions

In [4]:
def download_and_prep_data() -> Tuple[np.ndarray,np.ndarray,np.ndarray,np.ndarray]:
    """Download dataset and scale to [0, 1].

      Returns:
        tr_x: Training data.
        tr_y: Training labels.
        te_x: Testing data.
        te_y: Testing labels.
    """
    mnist_dataset = tf.keras.datasets.mnist
    (tr_x, tr_y), (te_x, te_y) = mnist_dataset.load_data()
    tr_x = tr_x / 255.0
    te_x = te_x / 255.0
    return tr_x, tr_y, te_x, te_y



def train_and_eval(model, input_data, num_epochs=10) -> float:
    """Returns model accuracy after train and evaluation."""
    tr_x, tr_y, te_x, te_y = input_data
    model.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])
    model.fit(tr_x, tr_y, epochs=num_epochs)
    _, test_acc = model.evaluate(te_x, te_y, verbose=2)
    return test_acc

# Symbolize Keras Objects

In [5]:
# NOTE(daiyip): We symbolize three Keras layers, so that their hyper-parameters
# can accept hyper values such as `pg.oneof`. Therefore, a search space for the
# model architecture can be represented as a hyper Sequential object.
Conv2D = pg.symbolize(tf.keras.layers.Conv2D)
Dense = pg.symbolize(tf.keras.layers.Dense)
MaxPool2D = pg.symbolize(tf.keras.layers.MaxPool2D)
Identity = pg.symbolize(tf.keras.initializers.Identity)
Sequential = pg.symbolize(tf.keras.Sequential)

# Test 1

In [ ]:
def nas_model():
    """NAS search space."""
    return Sequential(layers=pg.oneof([
          # Model family 1: only dense layers.
          [
              tf.keras.layers.Flatten(),
              # NOTE(daiyip): we use the symbolic Dense here as `pg.oneof` are
              # passed to its constructor to create a search space on Dense
              # hyper-parameters. On the next line, we use the regular Keras
              # Dense class since we don't tune its hyper-parameters, though
              # the symbolic Dense can also work on fixed hyper-parameter values.
              Dense(pg.oneof([64, 128]), activation=pg.oneof(['relu', 'sigmoid'])),
              tf.keras.layers.Dense(10, activation='softmax')
          ],
          # Model family 2: conv net.
          [
              tf.keras.layers.Lambda(lambda x: tf.reshape(x, (-1, 28, 28, 1))),
              Conv2D(filters=pg.oneof([64, 128]),
                     kernel_size=pg.oneof([(3, 3), (5, 5)]),
                     padding='same',
                     activation=pg.oneof(['relu', 'sigmoid'])),
              tf.keras.layers.Flatten(),
              tf.keras.layers.Dense(10, activation='softmax')
          ]
    ]))



# class MyDNAGenerator(pg.DNAGenerator):
#     def _propose(self):
#         return pg.DNA()[1,1]


def tune(max_trials, num_epochs):
    """Tune MNIST model via random search."""
    results = []
    input_data = download_and_prep_data()
    # NOTE(daiyip): `pg.sample` returns an iterator of (example, feedback_fn)
    # from a hyper object (the search space) and a DNAGenerator (the search
    # algorithm), with an optional flag to set the max examples to sample.
    # `example` is a materialized object of the search space, and `feedback_fn`
    # is a callable object that we can send back a float reward to the
    # controller. `feedback_fn` also has a property `dna` to access the DNA value
    # of current example.
    
    #my_gen = MyDNAGenerator()
    
    for model, feedback in pg.sample(nas_model(), pg.generators.Random(), max_trials):
        print('{}: DNA: {}'.format(feedback.id, feedback.dna))
        print(model)
        test_acc = train_and_eval(model, input_data, num_epochs)
        results.append((feedback.id, feedback.dna, test_acc))
        # NOTE: for random generator, following call to `feedback` is a no-op.
        # We keep it here in case we want to change algorithm.
        feedback(test_acc)
  
    # Print best results.
    top_results = sorted(results, key=lambda x: x[2], reverse=True)
    print('Top 10 results.')
    for i, (trial_id, dna, test_acc) in enumerate(top_results[:10]):
        print('#{0:2d} - trial {1:2d} ({2:.3f}): {3}'.format(i + 1, trial_id, test_acc, dna))


tune(3, 2)

# Test 2

In [ ]:
def nas_model():
    """NAS search space."""
    return Sequential(layers=pg.oneof([
          # Model family 1: only dense layers.
          [
              tf.keras.layers.Flatten(),
              # NOTE(daiyip): we use the symbolic Dense here as `pg.oneof` are
              # passed to its constructor to create a search space on Dense
              # hyper-parameters. On the next line, we use the regular Keras
              # Dense class since we don't tune its hyper-parameters, though
              # the symbolic Dense can also work on fixed hyper-parameter values.
              Dense(pg.oneof([64, 128]), activation=pg.oneof(['relu', 'sigmoid'])),
              tf.keras.layers.Dense(10, activation='softmax')
          ],
          # Model family 2: conv net.
          [
              tf.keras.layers.Lambda(lambda x: tf.reshape(x, (-1, 28, 28, 1))),
              Conv2D(filters=pg.oneof([64, 128]),
                     kernel_size=pg.oneof([(3, 3), (5, 5)]),
                     padding='same',
                     activation=pg.oneof(['relu', 'sigmoid'])),
              tf.keras.layers.Flatten(),
              tf.keras.layers.Dense(10, activation='softmax')
          ]
    ]))


def tune(max_trials, num_epochs):
    """Tune MNIST model via random search."""
    results = []
    input_data = download_and_prep_data()
    # NOTE(daiyip): `pg.sample` returns an iterator of (example, feedback_fn)
    # from a hyper object (the search space) and a DNAGenerator (the search
    # algorithm), with an optional flag to set the max examples to sample.
    # `example` is a materialized object of the search space, and `feedback_fn`
    # is a callable object that we can send back a float reward to the
    # controller. `feedback_fn` also has a property `dna` to access the DNA value
    # of current example.
    
    for model, feedback in pg.sample(nas_model(), pg.generators.Random(), max_trials):
        print('{}: DNA: {}'.format(feedback.id, feedback.dna))
        print(model)
        test_acc = train_and_eval(model, input_data, num_epochs)
        results.append((feedback.id, feedback.dna, test_acc))
        # NOTE: for random generator, following call to `feedback` is a no-op.
        # We keep it here in case we want to change algorithm.
        feedback(test_acc)
  
    # Print best results.
    top_results = sorted(results, key=lambda x: x[2], reverse=True)
    print('Top 10 results.')
    for i, (trial_id, dna, test_acc) in enumerate(top_results[:10]):
        print('#{0:2d} - trial {1:2d} ({2:.3f}): {3}'.format(i + 1, trial_id, test_acc, dna))


tune(3, 2)

In [13]:
@pg.symbolize
class A:
    def __init__(self, a, b):
        self.a = a
        self.b = b
    
    
    def _sum(self):
        return self.a + self.b
    

_a = A(1,2)
print(_a._sum())

_a.rebind(a=5)

print(_a._sum())

_a.rebind(a=10, b=20)
print(_a._sum())

3
7
30


In [41]:

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import RMSprop

@pg.symbolize
class Trainer(object):
    def __init__(self, model, optimizer):
        self.optimizer = optimizer
        self.model = model
    
    def train(self, input_data, num_epochs):    
        tr_x, tr_y, te_x, te_y = input_data
        
        print(f'self.optmizer: {self.optimizer}')
        print(f'self.model: {self.model}')
        
        self.model.compile(optimizer=self.optimizer, 
                           loss='sparse_categorical_crossentropy',
                           metrics=['accuracy'])
        
        self.model.fit(tr_x, tr_y, epochs=num_epochs)
        
        _, test_acc = self.model.evaluate(te_x, te_y, verbose=2)
        
        return test_acc

    
    
@pg.symbolize
class Stacked(object):
    def __init__(self, op, repeats):
        self.op = op
        self.repeats = repeats
    
    def __call__(self):
        print(f'Set op: {self.op} | Set repeats: {self.repeats}')
        return Sequential(layers=[tf.keras.layers.Lambda(lambda x: tf.reshape(x, (-1, 28, 28, 1)))]+\
                                 [self.op for _ in range(self.repeats)]+\
                                 [tf.keras.layers.GlobalAveragePooling2D()] +
                                 [tf.keras.layers.Dense(10, activation='softmax')])  
        


def create_trainer():
    #st = Stacked()
    model = Stacked(op=pg.oneof([Conv2D(pg.oneof([4, 8]), (3, 3)), 
                                 Conv2D(pg.oneof([4, 8]), (5, 5))]), 
                    repeats=3)
    return Trainer(model=model,
                   optimizer=pg.oneof([Adam(2e-2), RMSprop(pg.floatv(1e-6, 1e-3))]))


# def nas_model_2(repeats):
#     layers = [tf.keras.layers.Flatten()] + \
#              [Dense(pg.oneof([64, 128]), 
#                     activation=pg.oneof(['relu', 'sigmoid']))] * repeats + \
#              [tf.keras.layers.Dense(10, activation='softmax')]
#     return Sequential(layers=layers)


def tune(max_trials, num_epochs):
    """Tune MNIST model via random search."""
    results = []
    input_data = download_and_prep_data()
        
    for trainer, feedback in pg.sample(create_trainer(), pg.generators.Random(), max_trials, partition_fn=None):
        print('\n\n{}: DNA: {}'.format(feedback.id, feedback.dna))
        
        #print(model)
        #test_acc = train_and_eval(model, input_data, num_epochs)
        
        print(trainer)
        
        model_2 = trainer.model()
        
        #print(model_2)
        
        trainer.rebind(model=model_2)
        
        test_acc = trainer.train(input_data, num_epochs)
        
        results.append((feedback.id, feedback.dna, test_acc))
        
        feedback(test_acc)
  
    # Print best results.
    top_results = sorted(results, key=lambda x: x[2], reverse=True)
    print('Top 10 results.')
    for i, (trial_id, dna, test_acc) in enumerate(top_results[:10]):
        print('#{0:2d} - trial {1:2d} ({2:.3f}): {3}'.format(i + 1, trial_id, test_acc, dna))
        
        
tune(3,2)        

Ignoring keyword arguments that are not supported by 'in-memory' backend: {'partition_fn': None}




1: DNA: DNA([(1, 0), 0])
Trainer(
  model = Stacked(
    op = Conv2D(
      filters = 4,
      kernel_size = (5, 5),
      strides = (1, 1),
      padding = 'valid',
      data_format = None,
      dilation_rate = (1, 1),
      groups = 1,
      activation = None,
      use_bias = True,
      kernel_initializer = 'glorot_uniform',
      bias_initializer = 'zeros',
      kernel_regularizer = None,
      bias_regularizer = None,
      activity_regularizer = None,
      kernel_constraint = None,
      bias_constraint = None
    ),
    repeats = 3
  ),
  optimizer = <tensorflow.python.keras.optimizer_v2.adam.Adam object at 0x7efc7c4ba6d0>
)
Set op: Conv2D(
  filters = 4,
  kernel_size = (5, 5),
  strides = (1, 1),
  padding = 'valid',
  data_format = None,
  dilation_rate = (1, 1),
  groups = 1,
  activation = None,
  use_bias = True,
  kernel_initializer = 'glorot_uniform',
  bias_initializer = 'zeros',
  kernel_regularizer = None,
  bias_regularizer = None,
  activity_regularizer = Non

ValueError: in user code:

    /home/guilherme/data2/anaconda3/envs/icao_nets_training/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:855 train_function  *
        return step_function(self, iterator)
    /home/guilherme/data2/anaconda3/envs/icao_nets_training/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:845 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /home/guilherme/data2/anaconda3/envs/icao_nets_training/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:1285 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /home/guilherme/data2/anaconda3/envs/icao_nets_training/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:2833 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /home/guilherme/data2/anaconda3/envs/icao_nets_training/lib/python3.8/site-packages/tensorflow/py

# Test 3

In [10]:

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import RMSprop

@pg.symbolize
class Trainer(object):
    def __init__(self, model, optimizer):
        self.optimizer = optimizer
        self.model = model
    
    
def train(model, optimizer, input_data, num_epochs):    
    tr_x, tr_y, te_x, te_y = input_data

    print(f'self.optmizer: {optimizer}')
    print(f'self.model: {model}')

    model.compile(optimizer=optimizer, 
                       loss='sparse_categorical_crossentropy',
                       metrics=['accuracy'])

    model.fit(tr_x, tr_y, epochs=num_epochs)

    _, test_acc = model.evaluate(te_x, te_y, verbose=2)

    return test_acc
    
    
@pg.symbolize
class Stacked(object):
    def __init__(self, op, repeats):
        self.op = op
        self.repeats = repeats
    
    def __call__(self):
        print(f'Set op: {self.op} | Set repeats: {self.repeats}')
        return Sequential(layers=[tf.keras.layers.Lambda(lambda x: tf.reshape(x, (-1, 28, 28, 1)))]+\
                                 [self.op for _ in range(self.repeats)]+\
                                 [tf.keras.layers.GlobalAveragePooling2D()] +
                                 [tf.keras.layers.Dense(10, activation='softmax')])  
        


def create_trainer():
    #st = Stacked()
    model = Stacked(op=pg.oneof([Conv2D(pg.oneof([4, 8]), (3, 3)), 
                                 Conv2D(pg.oneof([4, 8]), (5, 5))]), 
                    repeats=4)
    return Trainer(model=model,
                   optimizer=pg.oneof([Adam(2e-2), RMSprop(pg.floatv(1e-6, 1e-3))]))


# def nas_model_2(repeats):
#     layers = [tf.keras.layers.Flatten()] + \
#              [Dense(pg.oneof([64, 128]), 
#                     activation=pg.oneof(['relu', 'sigmoid']))] * repeats + \
#              [tf.keras.layers.Dense(10, activation='softmax')]
#     return Sequential(layers=layers)


def tune(max_trials, num_epochs):
    """Tune MNIST model via random search."""
    results = []
    input_data = download_and_prep_data()
        
    for trainer, feedback in pg.sample(pg.hyper.trace(create_trainer), pg.generators.Random(), max_trials):
        print('\n\n{}: DNA: {}'.format(feedback.id, feedback.dna))
        
        #print(model)
        #test_acc = train_and_eval(model, input_data, num_epochs)
        
        with trainer() as t:
            print(t)
            
            test_acc = train(trainer.model, trainer.optimizer, input_data, num_epochs)

            results.append((feedback.id, feedback.dna, test_acc))

            feedback(test_acc)
  
    # Print best results.
    top_results = sorted(results, key=lambda x: x[2], reverse=True)
    print('Top 10 results.')
    for i, (trial_id, dna, test_acc) in enumerate(top_results[:10]):
        print('#{0:2d} - trial {1:2d} ({2:.3f}): {3}'.format(i + 1, trial_id, test_acc, dna))
        
        
tune(3,2)        



1: DNA: DNA([0, 0, 0, 0.0005197101697627721, 0])
None


AttributeError: 'function' object has no attribute 'model'

# Test 4

In [6]:
from tensorflow.keras.layers import Concatenate, Add


def get_m1():
    # NOTE(daiyip): We use a zero-argument `lambda` function to
    # wrap each candidate in order to construct a conditional search
    # space.
    return lambda: [  # pylint: disable=g-long-lambda
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(pg.oneof([64, 128]),
                                  pg.oneof(['relu', 'sigmoid']))
        ]



def get_m2():
    return lambda: [tf.keras.layers.Lambda(lambda x: tf.reshape(x, (-1, 28, 28, 1))),
                    tf.keras.layers.Conv2D(pg.oneof([64, 128]),
                                pg.oneof([(3, 3), (5, 5)]),
                                activation=pg.oneof(['relu', 'sigmoid'])),
                    tf.keras.layers.Flatten()]



# def get_m2(z):
#     l = [tf.keras.layers.Conv2D(pg.oneof([64, 128]),
#                                 pg.oneof([(3, 3), (5, 5)]),
#                                 activation=pg.oneof(['relu', 'sigmoid'])) for l in range(z)]
    
#     final_list = [tf.keras.layers.Lambda(lambda x: tf.reshape(x, (-1, 28, 28, 1)))] + \
#                  l + \
#                  [tf.keras.layers.Flatten()]
    
#     return lambda: final_list


def get_final_layer():
    return [tf.keras.layers.Dense(10, activation='softmax')]


# def create_model() -> tf.keras.Model:
#     """Create model for training.
 
#     Create a simple tf.keras model for training.
 
#     Returns:
#       The model to use for training.
#     """
#     left_branch = tf.keras.Sequential(pg.oneof([get_m1(), get_m2()]))
#     right_branch = tf.keras.Sequential(pg.oneof([get_m1(), get_m2()]))
#     merged = Concatenate([left_branch, right_branch])

#     model = Sequential()
#     model.add(merged)
#     model.add(get_final_layer())
    
#     return model
                                          

def create_model_orig() -> tf.keras.Model:
    return Sequential(pg.oneof([get_m1(), get_m2()]) + get_final_layer())


def train_and_eval(input_data, num_epochs) -> None:
    """Run training and evaluation."""
    tr_x, tr_y, te_x, te_y = input_data
    model = create_model_orig()
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    model.fit(tr_x, tr_y, epochs=num_epochs)
    test_loss, test_acc = model.evaluate(te_x, te_y, verbose=2)
    print('Test loss: {}, accuracy: {}'.format(test_loss, test_acc))
    return test_acc


def tune(max_trials, num_epochs):
    """Tune MNIST model via random search."""
    results = []
    input_data = download_and_prep_data()
    # NOTE(daiyip): `pg.sample` returns an iterator of (example, feedback_fn)
    # from a hyper object (the search space) and a DNAGenerator (the search
    # algorithm), with an optional flag to set the max examples to sample.
    # Different from defining a search space from symbolized classes or functors,
    # `pg.hyper.trace` allows users to define a search space
    # without symbolizing user classes, by eagerly executing the user function to
    # collect decision points in the search space. Each point in the search space
    # is a list of numbers materialized from the decision points in the search
    # space. Instead of using these numbers to construct the user program, we can
    # directly apply `feedback.dna` to the user function called in the loop via
    # `with example():`, which materializes the hyper values used
    # within the user function using the decisions made by the search algorithm
    # for current trial. As a result, each call to the user function yields
    # different hyper-parameters implicitly bound with current trial.
    for example, feedback in pg.sample(pg.hyper.trace(create_model_orig), pg.generators.Random(), max_trials):
        
        print('{}: DNA: {}'.format(feedback.id, feedback.dna))
        
        with example():
            test_acc = train_and_eval(input_data, num_epochs)
        
        results.append((feedback.id, feedback.dna, test_acc))
        
        # NOTE: for random generator, following call to `feedback` is a no-op.
        # We keep it here in case we want to change algorithm.
        feedback(test_acc)

    # Print best results.
    top_results = sorted(results, key=lambda x: x[2], reverse=True)
    print('Top 10 results.')
    for i, (trial_id, dna, test_acc) in enumerate(top_results[:10]):
        print('#{0:2d} - trial {1:2d} ({2:.3f}): {3}'.format(i + 1, trial_id, test_acc, dna))


tune(3, 2)

1: DNA: DNA(0, [1, 0])
Epoch 1/2
1875/1875 [==============================] - 4s 1ms/step - loss: 0.2594 - accuracy: 0.9255
Epoch 2/2
1875/1875 [==============================] - 2s 1ms/step - loss: 0.1140 - accuracy: 0.9664
313/313 - 0s - loss: 0.0978 - accuracy: 0.9704
Test loss: 0.09777722507715225, accuracy: 0.9703999757766724
2: DNA: DNA(0, [0, 1])
Epoch 1/2
1875/1875 [==============================] - 2s 1ms/step - loss: 0.4664 - accuracy: 0.8855
Epoch 2/2
1875/1875 [==============================] - 2s 1ms/step - loss: 0.2226 - accuracy: 0.9363
313/313 - 0s - loss: 0.1979 - accuracy: 0.9432
Test loss: 0.19792458415031433, accuracy: 0.9431999921798706
3: DNA: DNA(1, [1, 0, 0])
Epoch 1/2
1875/1875 [==============================] - 6s 2ms/step - loss: 0.1430 - accuracy: 0.9579
Epoch 2/2
1875/1875 [==============================] - 3s 2ms/step - loss: 0.0574 - accuracy: 0.9826
313/313 - 0s - loss: 0.0625 - accuracy: 0.9800
Test loss: 0.06245436519384384, accuracy: 0.980000019073486

# Test 5 - NAS-Bench

In [24]:
import os
import sys

if './nasbench/nasbench' not in sys.path:
    sys.path.insert(0, './nasbench/nasbench')


import time
from absl import app
from absl import flags

#from nasbench.nasbench import api
import numpy as np
import pyglove as pg

# Useful constants
INPUT = 'input'
OUTPUT = 'output'
CONV3X3 = 'conv3x3-bn-relu'
CONV1X1 = 'conv1x1-bn-relu'
MAXPOOL3X3 = 'maxpool3x3'
NUM_VERTICES = 7
EDGE_SPOTS = NUM_VERTICES * (NUM_VERTICES - 1) / 2   # Upper triangular matrix
OP_SPOTS = NUM_VERTICES - 2   # Input/output vertices are fixed
ALLOWED_OPS = [CONV3X3, CONV1X1, MAXPOOL3X3]
ALLOWED_EDGES = [0, 1]   # Binary adjacency matrix

DEFAULT_NAS_BENCH_108_EPOCHS_FILE = '/home/guilherme/data2/doutorado/nasbench-101/data/nasbench_only108.tfrecord'

@pg.symbolize([('ops', pg.typing.List(pg.typing.Str())), ('matrix', pg.typing.List(pg.typing.List(pg.typing.Int())))])
def model_spec(ops, matrix):
    """NASBench model spec that is parameterized by ops and their connections.
  
    Args:
      ops: a list of allowed ops except the INPUT and OUTPUT layer.
      matrix: the adjacency matrix for the connectivity of each layers, which
        should be an upper triangle matrix.
  
    Returns:
      A NASBench spec.
    """
    return api.ModelSpec(matrix=np.array(matrix), ops=[INPUT] + ops + [OUTPUT])


# We introduce hints so controller can deal with different knobs differently.
OP_HINTS = 0
EDGE_HINTS = 1


def default_search_space():
    """The default search space in NAS-Bench.
  
    This equals to the default search space of NAS-Bench, which mutate candidate
    ops and their connections.
  
    Returns:
      A hyper model object that repesents a search space.
    """
    matrix = [
        [pg.oneof([0, 1], hints=EDGE_HINTS) if y > x else 0
         for y in range(NUM_VERTICES)]
        for x in range(NUM_VERTICES)
    ]
    return model_spec(pg.manyof(NUM_VERTICES - 2, ALLOWED_OPS, choices_distinct=False, hints=OP_HINTS), matrix)


default_search_space()

model_spec(ops=ManyOf(name=None, hints=0, num_choices=5, candidates=[0: 'conv3x3-bn-relu', 1: 'conv1x1-bn-relu', 2: 'maxpool3x3'], choices_distinct=False, choices_sorted=False, where=None), matrix=[0: [0: 0, 1: OneOf(name=None, hints=1, num_choices=1, candidates=[0: 0, 1: 1], choices_distinct=True, choices_sorted=False, where=None), 2: OneOf(name=None, hints=1, num_choices=1, candidates=[0: 0, 1: 1], choices_distinct=True, choices_sorted=False, where=None), 3: OneOf(name=None, hints=1, num_choices=1, candidates=[0: 0, 1: 1], choices_distinct=True, choices_sorted=False, where=None), 4: OneOf(name=None, hints=1, num_choices=1, candidates=[0: 0, 1: 1], choices_distinct=True, choices_sorted=False, where=None), 5: OneOf(name=None, hints=1, num_choices=1, candidates=[0: 0, 1: 1], choices_distinct=True, choices_sorted=False, where=None), 6: OneOf(name=None, hints=1, num_choices=1, candidates=[0: 0, 1: 1], choices_distinct=True, choices_sorted=False, where=None)], 1: [0: 0, 1: 0, 2: OneOf(name

In [31]:
# def search(nasbench, search_model, algo, repeat_id, max_train_hours=5e6):
#     """Define the search procedure.
  
#     Args:
#       nasbench: NASBench object.
#       search_model: which is a `model` object annotated with `oneof`.
#       algo: algorithm for search.
#       repeat_id: identifier of current repeat.
#       max_train_hours: max time budget to train the models, which is the sum
#         of training time queried from NAS-Bench.
  
#     Returns:
#       A tuple of (total time spent at step i for all steps,
#                   best validation accuracy at step i for all steps,
#                   best test accuracy at step i for all steps)
#     """
#     #nasbench.reset_budget_counters()
#     times, best_valids, best_tests = [0.0], [0.0], [0.0]
#     valid_models = 0
#     time_spent = 0
#     start_time = time.time()
#     last_report_time = start_time
#     for model, feedback in pg.sample(search_model, algo, name=str(repeat_id)):
        #spec = model()
        #if nasbench.is_valid(spec):
        #    results = nasbench.query(spec)
        #    valid_models += 1
        #    feedback(results['validation_accuracy'])
        #    if results['validation_accuracy'] > best_valids[-1]:
        #        best_valids.append(results['validation_accuracy'])
        #        best_tests.append(results['test_accuracy'])
        #    else:
        #        best_valids.append(best_valids[-1])
        #        best_tests.append(best_tests[-1])
        #    time_spent, _ = nasbench.get_budget_counters()
        #    times.append(time_spent)
        #    if time_spent > max_train_hours:
        #      # Break the first time we exceed the budget.
        #        feedback.end_loop()
        #        break
        #else:
        #    feedback.skip()
        
        #print(model)
    
        #if feedback.id % 100 == 0:
        #    now = time.time()
        #    print(f'Tried {feedback.id} models, valid {valid_models}, '
        #          f'time_spent {time_spent}, elapse since last report: '
        #          f'{now - last_report_time} seconds.')
        #    last_report_time = now
    #print(f'Total time elapse: {time.time() - start_time} seconds.')
    #return times, best_valids, best_tests


@pg.symbolize
def node_selector(x, hints):
    """A functor to select node based on hints."""
    return x.spec.hints == hints


def create_search_algorithm(flag_value):
    """Create search algorithm from flag."""
    if flag_value == 'random':
        return pg.generators.Random()
    elif flag_value == 'evolution':
        return pg.evolution.regularized_evolution(
          mutator=(
              pg.evolution.mutators.Uniform(
                  where=node_selector(hints=OP_HINTS))         # pylint: disable=no-value-for-parameter
              >> pg.evolution.mutators.Uniform(
                  where=node_selector(hints=EDGE_HINTS)) ** 3  # pylint: disable=no-value-for-parameter
          ),
          population_size=50,
          tournament_size=10)
    else:
        return pg.load(flag_value)


# Load the dataset.
#nasbench = api.NASBench(DEFAULT_NAS_BENCH_108_EPOCHS_FILE)

search_model = default_search_space()
  
#print(search_model)    
    
# Start search.
#for i in range(FLAGS.repeat_start, FLAGS.repeat_end):
#    print(f'Repeat #{i}')
    # Create algorithm.
algorithm = create_search_algorithm('random')

print(algorithm)

for model, feedback in pg.sample(search_model, algorithm, name='1'):
    spec = model()
    
    #times, best_valid, best_test = search(nasbench, search_model, algorithm, i, 1)
  
    #print('%15s %15s %15s %15s' % ('# trials','best valid','best test','simulated train hours'))
    #print('%15d %15.4f %15.4f %15d' % (len(times), best_valid[-1], best_test[-1], times[-1]))
  
    # if FLAGS.output_dir:
    #     pg.Dict(times=times, best_valid=best_valid, best_test=best_test).save(os.path.join(FLAGS.output_dir, f'repeat_{i}.json'))




Random(
  seed = None
)


NameError: name 'api' is not defined

# Test 6 - NATS-Bench

In [20]:
import os
import time
from absl import app
from absl import flags

import nats_bench
import pyglove as pg


DEFAULT_NATS_FILEs = dict(tss=None, sss=None)

# Results in the paper use reporting epochs $H^1$ and $H^2$ for the topology
# and size search spaces respectively. See section 3.3 of the paper.
DEFAULT_REPORTING_EPOCH = dict(tss=200, sss=90)
VALIDATION_SET_REPORTING_EPOCH = 12


@pg.functor([('ops', pg.typing.List(pg.typing.Str())),('num_nodes', pg.typing.Int())])
def model_tss_spc(ops, num_nodes):
    """The architecture in the topology search space of NATS-Bench."""
    nodes, k = [], 0
    for i in range(1, num_nodes):
        xstrs = []
        for j in range(i):
            xstrs.append('{:}~{:}'.format(ops[k], j))
            k += 1
        nodes.append('|' + '|'.join(xstrs) + '|')
    return '+'.join(nodes)


@pg.functor([('channels', pg.typing.List(pg.typing.Int()))])
def model_sss_spc(channels):
    """The architecture in the size search space of NATS-Bench."""
    return ':'.join(str(x) for x in channels)


def get_search_space(ss_indicator):
    """The default search space in NATS-Bench.
  
    Args:
      ss_indicator: tss or sss, indicating the topology or size search space.
  
    Returns:
      A hyper model object that repesents a search space.
    """
    info = nats_bench.search_space_info('nats-bench', ss_indicator)
    print(info)
    if ss_indicator == 'tss':
        total = info['num_nodes'] * (info['num_nodes'] - 1) // 2
        return model_tss_spc(pg.sublist_of(total, info['op_names'], choices_distinct=False), info['num_nodes'])
    elif ss_indicator == 'sss':
        return model_sss_spc(pg.sublist_of(info['num_layers'], info['candidates'], choices_distinct=False))


def search(nats_api,
           search_model,
           algo,
           dataset='cifar10',
           reporting_epoch=12,
           max_train_hours=2e4):
    """Define the search procedure.
  
    Args:
      nats_api: the NATS-Bench object.
      search_model: which is a `model` object annotated with `one_of`.
      algo: algorithm for search.
      dataset: the target dataset
      reporting_epoch: Use test set results for models trained for this many epochs.
      max_train_hours: max time budget to train the models, which is the sum of training time queried from NAS-Bench.
  
    Returns:
      A tuple of (total time spent at step i for all steps,
                  best validation accuracy at step i for all steps,
                  best test accuracy at step i for all steps)
    """
    nats_api.reset_time()
    times, best_valids, best_tests = [0.0], [0.0], [0.0]
    valid_models = 0
    time_spent = 0
    start_time = time.time()
    last_report_time = start_time
    for model, feedback in pg.sample(search_model, algo):
        spec = model()
        
        print(spec)
        
        (validation_accuracy, _, _, _) = nats_api.simulate_train_eval(spec, dataset=dataset, hp=VALIDATION_SET_REPORTING_EPOCH)
        
        time_spent = nats_api.used_time
        
        more_info = nats_api.get_more_info(spec, dataset, hp=reporting_epoch)  # pytype: disable=wrong-arg-types  # dict-kwargs
        
        valid_models += 1
        
        feedback(validation_accuracy)
        
        if validation_accuracy > best_valids[-1]:
            best_valids.append(validation_accuracy)
            best_tests.append(more_info['test-accuracy'])
        else:
            best_valids.append(best_valids[-1])
            best_tests.append(best_tests[-1])

        times.append(time_spent)
        time_spent_in_hours = time_spent / (60 * 60)
        
        if time_spent_in_hours > max_train_hours:
            break # Break the first time we exceed the budget.
        
        if feedback.id % 100 == 0:
            now = time.time()
            print(f'Tried {feedback.id} models, valid {valid_models}, '
                  f'time_spent_in_hours: {int(time_spent_in_hours)}h, '
                  f'time_spent: {round(time_spent,3)}s, '
                  f'elapse since last report: {round(now - last_report_time,3)}s.')
            last_report_time = now
            
    print(f'Total time elapse: {time.time() - start_time} seconds.')
    # Remove the first element of each list because these are placeholders
    # used for computing the current max. They don't correspond to
    # actual results from nats_api.
    return times[1:], best_valids[1:], best_tests[1:]


def get_algorithm(algorithm_str):
    """Creates algorithm."""
    if algorithm_str == 'random':
        return pg.generators.Random()
    elif algorithm_str == 'evolution':
        return pg.evolution.regularized_evolution(mutator=pg.evolution.mutators.Uniform(), population_size=50, tournament_size=10)
    else:
        return pg.load(algorithm_str)


SEARCH_SPACE = 'sss'    
    
# Load the dataset.
nats_bench.api_utils.reset_file_system('default')
nats_api = nats_bench.create(DEFAULT_NATS_FILEs[SEARCH_SPACE], SEARCH_SPACE, fast_mode=True, verbose=False)

# Create search space.
search_model = get_search_space(SEARCH_SPACE)
reporting_epoch = DEFAULT_REPORTING_EPOCH[SEARCH_SPACE]

algorithm = get_algorithm('evolution')

times, best_valid, best_test = search(nats_api, search_model, algorithm, 'cifar10', reporting_epoch, max_train_hours=1)

print('%15s %15s %15s %15s' % ('# trials', 'best valid (%)', 'best test (%)', 'simulated train hours'))
print('%15d %15.4f %15.4f %21d' % (len(times), best_valid[-1], best_test[-1], times[-1]))

[2023-04-13 01:49:25] Try to use the default NATS-Bench (size) path from fast_mode=True and path=None.
{'candidates': [8, 16, 24, 32, 40, 48, 56, 64], 'num_layers': 5}
56:16:16:64:16
48:64:32:40:16
64:8:24:48:32
40:16:40:8:40
32:24:48:56:16
40:64:16:40:56
32:8:64:32:8
16:32:40:48:8
32:56:64:40:40
56:16:24:32:48
56:40:40:56:24
32:40:24:32:56
40:64:48:56:8
56:24:56:32:48
24:16:56:40:24
16:16:16:16:8
56:24:64:32:40
24:40:24:56:24
32:16:64:48:16
8:8:40:32:32
56:40:48:48:48
16:48:16:40:64
24:56:56:24:8
56:48:40:48:40
40:32:56:16:16
16:48:16:48:8
8:40:56:48:48
24:24:8:64:8
32:40:48:48:8
16:8:8:32:48
56:32:32:8:32
8:64:32:56:32
56:32:16:40:40
24:24:24:40:8
8:24:16:8:32
16:8:64:40:40
64:64:48:40:32
8:16:8:40:56
24:32:56:48:56
48:48:32:48:56
40:24:40:16:32
48:8:56:32:32
8:8:24:32:16
16:8:16:40:48
56:32:16:40:64
40:8:8:24:24
48:56:8:56:56
48:8:24:40:56
64:64:8:8:8
24:64:56:24:16
56:48:16:40:64
24:32:56:40:56
32:56:32:40:40
64:64:48:40:40
48:16:56:40:48
64:32:48:16:32
56:32:40:48:40
24:16:56:48:5

# Test 7 


In [55]:
import os
import time
import random
from absl import app
from absl import flags

import nats_bench
import pyglove as pg


DEFAULT_NATS_FILEs = dict(tss=None, sss=None)

# Results in the paper use reporting epochs $H^1$ and $H^2$ for the topology
# and size search spaces respectively. See section 3.3 of the paper.
DEFAULT_REPORTING_EPOCH = dict(tss=200, sss=90)
VALIDATION_SET_REPORTING_EPOCH = 12


@pg.functor([('channels', pg.typing.List(pg.typing.Int()))])
def model_sss_spc(channels):
    print(channels)
    """The architecture in the size search space of NATS-Bench."""
    return ':'.join(str(x) for x in channels)


@pg.functor([('n_denses', pg.typing.List(pg.typing.Int()))])
def model_1_spc(n_denses):
    print(n_denses)
    return {f'n_denses_{idx}':x for idx,x in enumerate(n_denses)}


def get_search_space(ss_indicator):
    """The default search space in NATS-Bench.
  
    Args:
      ss_indicator: tss or sss, indicating the topology or size search space.
  
    Returns:
      A hyper model object that repesents a search space.
    """
    #info = nats_bench.search_space_info('nats-bench', ss_indicator)
    #print(info)  # 'candidates': [8, 16, 24, 32, 40, 48, 56, 64], 'num_layers': 5
    if ss_indicator == 'tss':
        total = info['num_nodes'] * (info['num_nodes'] - 1) // 2
        return model_tss_spc(pg.sublist_of(total, info['op_names'], choices_distinct=False), info['num_nodes'])
    elif ss_indicator == 'sss':
        #return model_sss_spc(pg.sublist_of(info['num_layers'], info['candidates'], choices_distinct=False))
        return model_sss_spc(pg.sublist_of(5, [8, 16, 24, 32, 40, 48, 56, 64], choices_distinct=False))
    elif ss_indicator == 'ss_1':
        return model_1_spc(pg.sublist_of(4, [1, 2, 3, 4, 5], choices_distinct=False))
    


def sim_train_eval(model):
    return random.random()
    
    
    
def search(nats_api,
           search_model,
           algo,
           dataset='cifar10',
           reporting_epoch=12,
           max_train_hours=2e4):
    """Define the search procedure.
  
    Args:
      nats_api: the NATS-Bench object.
      search_model: which is a `model` object annotated with `one_of`.
      algo: algorithm for search.
      dataset: the target dataset
      reporting_epoch: Use test set results for models trained for this many epochs.
      max_train_hours: max time budget to train the models, which is the sum of training time queried from NAS-Bench.
  
    Returns:
      A tuple of (total time spent at step i for all steps,
                  best validation accuracy at step i for all steps,
                  best test accuracy at step i for all steps)
    """
    #nats_api.reset_time()
    times, best_valids, best_tests = [0.0], [0.0], [0.0]
    valid_models = 0
    time_spent = 0
    start_time = time.time()
    last_report_time = start_time
    cnt = 0
    for model, feedback in pg.sample(search_model, algo):
        spec = model()
        
        print(f'Architecture {cnt}: {spec}')
        
        cnt += 1
        
        if cnt == 4:
            break
        
#         (validation_accuracy, _, _, _) = nats_api.simulate_train_eval(spec, dataset=dataset, hp=VALIDATION_SET_REPORTING_EPOCH)
        validation_accuracy = sim_train_eval(spec)
    
#         time_spent = nats_api.used_time
        
#         more_info = nats_api.get_more_info(spec, dataset, hp=reporting_epoch)  # pytype: disable=wrong-arg-types  # dict-kwargs
        
#         valid_models += 1
        
        feedback(validation_accuracy)
        
#         if validation_accuracy > best_valids[-1]:
#             best_valids.append(validation_accuracy)
#             best_tests.append(more_info['test-accuracy'])
#         else:
#             best_valids.append(best_valids[-1])
#             best_tests.append(best_tests[-1])

#         times.append(time_spent)
#         time_spent_in_hours = time_spent / (60 * 60)
        
#         if time_spent_in_hours > max_train_hours:
#             break # Break the first time we exceed the budget.
        
#         if feedback.id % 100 == 0:
#             now = time.time()
#             print(f'Tried {feedback.id} models, valid {valid_models}, '
#                   f'time_spent_in_hours: {int(time_spent_in_hours)}h, '
#                   f'time_spent: {round(time_spent,3)}s, '
#                   f'elapse since last report: {round(now - last_report_time,3)}s.')
#             last_report_time = now
            
    # print(f'Total time elapse: {time.time() - start_time} seconds.')
    # Remove the first element of each list because these are placeholders
    # used for computing the current max. They don't correspond to
    # actual results from nats_api.
    # return times[1:], best_valids[1:], best_tests[1:]


def get_algorithm(algorithm_str):
    """Creates algorithm."""
    if algorithm_str == 'random':
        return pg.generators.Random()
    elif algorithm_str == 'evolution':
        return pg.evolution.regularized_evolution(mutator=pg.evolution.mutators.Uniform(), population_size=50, tournament_size=10)
    else:
        return pg.load(algorithm_str)


ss_indicator = 'ss_1'    
    
# Load the dataset.
#nats_bench.api_utils.reset_file_system('default')
#nats_api = nats_bench.create(DEFAULT_NATS_FILEs[ss_indicator], ss_indicator, fast_mode=True, verbose=False)
nats_api = None
                           
# Create search space.
search_model = get_search_space(ss_indicator)

algorithm = get_algorithm('evolution')

#times, best_valid, best_test = search(nats_api, search_model, algorithm, dataset='cifar10', max_train_hours=0.05)
search(nats_api, search_model, algorithm, dataset='cifar10', max_train_hours=0.05)

#print('%15s %15s %15s %15s' % ('# trials', 'best valid (%)', 'best test (%)', 'simulated train hours'))
#print('%15d %15.4f %15.4f %21d' % (len(times), best_valid[-1], best_test[-1], times[-1]))

[
  0 : 2,
  1 : 5,
  2 : 1,
  3 : 4
]
Architecture 0: {'n_denses_0': 2, 'n_denses_1': 5, 'n_denses_2': 1, 'n_denses_3': 4}
[
  0 : 2,
  1 : 5,
  2 : 4,
  3 : 5
]
Architecture 1: {'n_denses_0': 2, 'n_denses_1': 5, 'n_denses_2': 4, 'n_denses_3': 5}
[
  0 : 2,
  1 : 2,
  2 : 5,
  3 : 3
]
Architecture 2: {'n_denses_0': 2, 'n_denses_1': 2, 'n_denses_2': 5, 'n_denses_3': 3}
[
  0 : 4,
  1 : 2,
  2 : 4,
  3 : 3
]
Architecture 3: {'n_denses_0': 4, 'n_denses_1': 2, 'n_denses_2': 4, 'n_denses_3': 3}
